# Adding Memory to a Chain (Part 2): Creating the Chain

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_ollama import ChatOllama

from langchain.memory import ConversationSummaryMemory

from operator import itemgetter

In [2]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [3]:
chat = ChatOllama(model = 'llama3.2',
                  temperature = 0,
                  num_predict=100)

In [5]:
chat_memory = ConversationSummaryMemory(llm = ChatOllama(model="llama3.2"),
                                        memory_key="message_log")

C:\Users\Eda AYDIN\AppData\Local\Temp\ipykernel_44736\3822549391.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatOllama(model="llama3.2"),


In [6]:
question = "Can you give me an interesting fact I probably didn't know about?"
# question = "Can you elaborate a bit more on this fact?"

In [7]:
dictionary_output = RunnablePassthrough.assign(
    message_log = RunnableLambda(chat_memory.load_memory_variables) | 
    itemgetter('message_log')).invoke(
    {'question':question})

In [8]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [9]:
ai_message_output = chat.invoke(prompt_value_output)

In [10]:
response = StrOutputParser().invoke(ai_message_output)

In [11]:
chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

In [12]:
chat_memory.load_memory_variables({})

{'message_log': 'Current summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\n\nNew lines of conversation:\nHuman: Can you give me an interesting fact I probably didn\'t know about?\nAI: I\'d be delighted to share something fascinating with you! Did you know that there is a type of jellyfish that is immortal? The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically immortal! Isn\'t that mind-blowing?\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. Additionally, the AI shares an interesting fact ab

In [13]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) | 
        itemgetter('message_log')) 
    | prompt_template 
    | chat 
    | StrOutputParser()
)

question = "Can you elaborate a bit more on this fact?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

response

"I'd be happy to dive deeper into the fascinating world of Turritopsis dohrnii! The process of transdifferentiation is quite complex and still not fully understood by scientists. Essentially, when the jellyfish reaches its adult stage, it can transform its cells into a younger state through a series of molecular transformations.\n\nFor example, if the jellyfish's adult form is damaged or needs to regenerate, it can trigger this process, which allows it to revert back to its polyp stage. From"

In [14]:
@chain
def memory_chain(question):
    
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) | 
            itemgetter('message_log')) 
        | prompt_template 
        | chat 
        | StrOutputParser()
    )
    
    chain1.get_graph().print_ascii()

    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question}, 
                             outputs = {'output':response})

    return response

In [15]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")

PydanticUserError: `load_memory_variables_input` is not fully defined; you should define `Dict`, then call `load_memory_variables_input.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.9/u/class-not-fully-defined